In [ ]:
object_name = "OB0001"

# Reducing Images

The goal of this notebook is to combine bias and flatfield images and apply these corrections to other images for each observing run of the Yasone catalogue. 

In each directy OB0001-9, this notebook will read in all the image files contained within the `raw` directory as provided by Julen. We will then trim all images as suggested by the fits header (into a mirrored `trimmed` directory). We then combine the bias images and create a main bias frame (stored in the `reduced` directory), from which we derive the combined flat fields for each band (g, r, i). Finally, we subtract the bias and divide by the flat field to produce initially reduced images. 


This notebook is heavily based on the guide https://ccdproc.readthedocs.io/en/latest/image_combination.html and uses ccdproc.

In [ ]:
from pathlib import Path
import os

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

import arya # my style files

In [ ]:
from convenience_functions import show_image, show_images, show_image_residual

In [ ]:
from astropy.nddata import CCDData
from astropy.io import fits
from astropy.stats import mad_std

import ccdproc
from ccdproc import ImageFileCollection

In [ ]:
plt.rcParams["figure.dpi"] = 150

# Paths

In [ ]:
filters = ["Sloan_g", "Sloan_r", "Sloan_i"]

In [ ]:
DATADIR = Path(f"../imaging/{object_name}")

In [ ]:
FIGDIR = DATADIR / "figures"
if not FIGDIR.is_dir():
    FIGDIR.mkdir()

def savefig(filename):
    plt.savefig(FIGDIR / f"{filename}.pdf")
    plt.savefig(FIGDIR / f"{filename}.png")

In [ ]:
"the order of lists of image groups"
allimgs_order = ["bias", "flat", "std", "obj"]

In [ ]:
def get_paths(dirname, data_dir=DATADIR):
    """Given the image superdirectory, returns the subdirectories corresponding to the 
    bias, flat, standard, and object images respectively
    """
    bias_dir = data_dir.joinpath(dirname, "bias")
    flat_dir = data_dir.joinpath(dirname, "flat")
    std_dir = data_dir.joinpath(dirname, "stds")
    obj_dir = data_dir.joinpath(dirname, "object")

    return bias_dir, flat_dir, std_dir, obj_dir

In [ ]:
def get_image_files(dirname):
    """Given an image superdirectory, retrieve ImageFileCollections
    for each of the subdirectories of images: bias, flat, stds, and object
    """
    paths = get_paths(dirname)
    return [ImageFileCollection(path) for path in paths]

In [ ]:
allimgs = get_image_files("raw")
imgs_bias, imgs_flat, imgs_std, imgs_obj = allimgs

### Plots

In [ ]:
for img, file in imgs_bias.data(return_fname=True):
    show_image(img, clabel="adu")
    plt.title(file)

The above plots show the bias reads

## Cleaning old files (if present)

In [ ]:
def get_new_path(filename, olddir, newdir):
    filename_new = filename.replace(olddir, newdir)

    if (newdir not in filename_new):
        print("Error creating new filename for", filename)
        return
        
    return Path(filename_new)

In [ ]:
for imgs in allimgs:
    for filename in imgs.files_filtered(include_path=True):
        for dirname in ["trimmed", "unbiased", "reduced"]:
            filename_new = get_new_path(filename, "raw", dirname)
            if filename_new.is_file():
                os.remove(filename_new)


# special files
filename = DATADIR / f"reduced/bias_combined.fits"
if filename.is_file():
    os.remove(filename)
    
for filtername in filters:
    filename = DATADIR / f"reduced/flat_combined_{filtername}.fits"
    if filename.is_file():
        os.remove(filename)

## Utilities

In [ ]:
def get_ccd_image(filename, **kwargs):
    """Return a CCDData object given the filename of a fits ccd image"""
    return CCDData.read(filename, **kwargs)

In [ ]:
def get_pathnames(image_collection):
    """Return the pathnames of each image in an image_collection"""
    return image_collection.files_filtered(include_path=True)

In [ ]:
def get_images(image_collection, **kwargs):
    """Retrieve each image from an image collection"""
    return image_collection.ccds(**kwargs)

# Image trimming and overscan

In [ ]:
IMG_SELECTION = "[28:2030,230:2026]" # taken from the CCD/fits header, also used by sausero, with exception of right boundary based on bias discontinuity and vignetting
OVERSCAN_SELECTION = "[9:24,1:2056]"

## Optional, overscan plots

In [ ]:
def get_overscan(image):
    return ccdproc.trim_image(CCDData(image), fits_section=OVERSCAN_SELECTION)

In [ ]:
def overscan_bias(image):
    return np.median(get_overscan(image), axis=1)

In [ ]:
for i in range(4):
    imgs = allimgs[i]

    for filename in get_pathnames(imgs):
        img = get_ccd_image(filename, unit="adu")
        bias = overscan_bias(img)
    
        plt.plot(bias, color=arya.COLORS[i])

    plt.plot([], color=arya.COLORS[i], label=allimgs_order[i])

plt.legend()
plt.xlabel("image row")
plt.ylabel("median overscan value")
savefig("overscan_biases")

Pixels in row 10:24 along all columns are useful for overscan, although maybe not useful since we will model sky background anyways.
Neglecting the overscan leaves a <30 adu background variation between images

In [ ]:
for img in imgs_flat.data():
    plt.plot(np.mean(img.data, axis=0))
plt.xlim(0,30)
plt.ylim(3000, 3200)
plt.xlabel("image column / pixel")
plt.ylabel("column mean value / adu")
plt.text(10, 3170, "overscan region", color="red")
plt.axvline(10, color="red")
plt.axvline(23, color="red")
plt.title("flats")
savefig("overscan_region_flats")

In [ ]:
IMG_SELECTION

The above plot shows bins in the column direction, illustrating where the overscan region is (columns 10-24). Before column 10, the response is nonlinear, and after, the light from the flat increases the values

In [ ]:
for img in imgs_bias.data():
    plt.plot(np.mean(img.data, axis=0))
plt.xlabel("image column / pixel")
plt.ylabel("column mean value / adu")
plt.xlim(2000, 2073)
plt.ylim(3120)
plt.axvline(28, color="red")
plt.axvline(2060, color="red")
plt.axvline(2030, color="blue")
plt.title("flats")


The above plot zooms in on the other side of the ccd. There is a discontinuity around column 2035 to be weary of within the trim region (left of red line)

## Triming and saving images

Since we elect to not consider the overscan (we will subtract a model sky background on a later step, so this step does not improve our pipeline), we simply trim the images in this step.

In [ ]:
def trim_image(image):
    return ccdproc.trim_image(CCDData(image), fits_section=IMG_SELECTION)

In [ ]:
img = CCDData.read(imgs_flat.files_filtered(include_path=True)[0], unit="adu")

In [ ]:
IMG_SELECTION

In [ ]:
def make_new_path(filename, olddir, newdir):
    filename_new = filename.replace(olddir, newdir)

    if (newdir not in filename_new):
        print("Error creating reduced filename for", filename)
        return
        
    print("processing ", filename, "\t => \t", filename_new)
    Path(filename_new).parent.mkdir(parents=True, exist_ok=True)
    return Path(filename_new)

In [ ]:
for i in range(4):
    imgs = allimgs[i]

    for filename in get_pathnames(imgs):
        img = get_ccd_image(filename, unit="adu")
        img_trimmed = trim_image(img)

        filename_new = make_new_path(filename, "raw", "trimmed")
        img_trimmed.write(filename_new)

### Plots

In [ ]:
img = next(get_images(imgs_flat, ccd_kwargs={"unit": "adu"}))

In [ ]:
show_image(img)
plt.axvline(28, color="red")
plt.axvline(2030, color="red")
plt.axhline(230, color="red")
plt.axhline(2026, color="red")

The red box shows the fits suggested trim region, which avoids the overscan (left), large vignetting (bottom, top, right)

In [ ]:
show_image(trim_image(img))

The trimmed plat image appears more reasonable, only more mild (70ish percent) vignetting towards the right edge and upper corner. We can also pick out dust features and uneven illumination now.

# Main Bias

In [ ]:
allimgs_trimmed = get_image_files("trimmed")

In [ ]:
def combine_images(filenames, **kwargs):
    return ccdproc.combine(filenames, 
                           method="median",
                           sigma_clip=True, sigma_clip_low_thresh=5, sigma_clip_high_thresh=5, 
                           sigma_clip_func=np.ma.median, sigma_clip_dev_func=mad_std, 
                           mem_limit=1e9,
                           **kwargs
                            )

In [ ]:
bias_main = combine_images(allimgs_trimmed[0].files_filtered(include_path=True))

In [ ]:
dirname = (DATADIR / "reduced")
if not dirname.is_dir():
    dirname.mkdir()

In [ ]:
bias_main.write(DATADIR / "reduced/bias_combined.fits")

### Plots
these below plots show the (trimmed) combined bias and a comparison of an unstacked bias frame to the combined bias frame

In [ ]:
show_image(bias_main)

In [ ]:
show_images([next(get_images(allimgs_trimmed[0])), bias_main], ["individual frame", "combined"])

In [ ]:
for img, file in allimgs_trimmed[0].data(return_fname=True):
    show_image_residual(img, bias_main, clabel="residual / adu")
    plt.title(file)
    savefig("bias_residual" + file)

The above images show the difference between the bias frames and the combined bias frame. As expected, the residuals show some random noise and some differences in each row, likely from row noise.

# Bias correction

In [ ]:
def subtract_bias(img, bias=bias_main):
    return ccdproc.subtract_bias(img, bias)

In [ ]:
for i in range(4):
    imgs = allimgs_trimmed[i]

    for filename in get_pathnames(imgs):
        img = get_ccd_image(filename)
        img_unbiased = subtract_bias(img)
        filename_new = make_new_path(filename, "trimmed", "unbiased")
        img_unbiased.write(filename_new)

# Main flat field

In [ ]:
allimgs_unbiased = get_image_files("unbiased")

In [ ]:
x, imgs_flat_unbiased, imgs_std_unbiased, imgs_obj_unbiased = allimgs_unbiased

In [ ]:
# Check that the only filters used are g r i
assert np.unique(imgs_flat_unbiased.summary["filter1"]) == ["OPEN"]
assert np.unique(imgs_flat_unbiased.summary["filter3"]) == ["OPEN"]
assert np.unique(imgs_flat_unbiased.summary["filter4"]) == ["OPEN"]
assert set(np.unique(imgs_flat_unbiased.summary["filter2"])) == set(filters)

In [ ]:
def flat_scale(A):
    return 1 / np.median(A)

In [ ]:
for filtername in filters:
    filenames = imgs_flat_unbiased.files_filtered(filter2=filtername, include_path=True)
    flat_combined = combine_images(filenames, scale=flat_scale)
    flat_combined.meta["combined"] = True

    filename_new = DATADIR / f"reduced/flat_combined_{filtername}.fits"
    filename_new.parent.mkdir(exist_ok=True)
    flat_combined.write(filename_new)

In [ ]:
flat_g = CCDData.read(DATADIR / "reduced/flat_combined_Sloan_g.fits")
flat_r = CCDData.read(DATADIR / "reduced/flat_combined_Sloan_r.fits")
flat_i = CCDData.read(DATADIR / "reduced/flat_combined_Sloan_i.fits")

In [ ]:
flats_reduced = {
    "Sloan_g": flat_g,
    "Sloan_r": flat_r,
    "Sloan_i": flat_i,
}

### Plots

In [ ]:
for (file, name) in allimgs_unbiased[1].ccds(return_fname=True):
    show_image(file)
    plt.title(name)

The above plot shows all flatfield frames

In [ ]:
imgs_flat_unbiased.summary

In [ ]:
for i, filtername in enumerate(filters):
    imgs = imgs_flat_unbiased.files_filtered(filter2=filtername, include_path=True)
    median_value = [np.median(data) for data in imgs_flat_unbiased.data(filter2=filtername)]
    mean_value = [np.mean(data) for data in imgs_flat_unbiased.data(filter2=filtername)]

    plt.plot(median_value, color=["g", "r", "y"][i], label=filters[i] + " median")
    plt.plot(mean_value, color=["g", "r", "y"][i], linestyle="--", label=filters[i] + " mean")

arya.Legend(-1)
plt.ylabel("central value / adu")
plt.xlabel("exposure number")
plt.title("flat fields")
savefig("flatfield_normalizations")

The plot above shows the global mean and median for each flatfield in each color. There is some change to the overal image scale, but both mean and median closely track eachother

In [ ]:
show_image(flat_g)
show_image(flat_r)
show_image(flat_i)

The above plot shows the combined flatfields normalized

In [ ]:
for filtername in filters:
    imgs = imgs_flat_unbiased.ccds(filter2=filtername, return_fname=True)
    for img, fname in imgs:
        flat = flats_reduced[img.header["filter2"]]
        img_reduced = img.data * flat_scale(img) / flat
        fig, axs = plt.subplots(1, 2, figsize=(5, 2.5))
        
        show_image(img_reduced, fig=fig, ax=axs[0], clabel = "flat / flat mean")
        
        axs[0].set_title(img.header["filter2"])
        plt.sca(axs[1])
        plt.hist(img_reduced.data.flatten())
        plt.yscale("log")
        plt.xlabel("relative value")
        plt.ylabel("pixel count")
        plt.tight_layout()
        savefig("flat_residual." + fname)

# Fully corrected frames

In [ ]:
for i in range(1, 4):
    imgs = allimgs_unbiased[i]

    for filename in get_pathnames(imgs):
        img = get_ccd_image(filename)
        flat = flats_reduced[img.header["filter2"]]
        img_reduced = ccdproc.flat_correct(img, flat)
        filename_new = make_new_path(filename, "unbiased", "reduced")
        img_reduced.write(filename_new)

### Plots
some last plots of various frames

In [ ]:
imgs_std_reduced = ImageFileCollection(DATADIR / "reduced/stds")

In [ ]:
imgs_obj_reduced = ImageFileCollection(DATADIR / "reduced/object")

In [ ]:
ccds_obj_raw = get_images(imgs_obj, ccd_kwargs={"unit": "adu"}, return_fname=True)
ccds_obj_trimmed = get_images(allimgs_trimmed[-1], return_fname=True)
ccds_obj_unbiased = get_images(allimgs_trimmed[-1], return_fname=True)

for img, name in imgs_obj_reduced.ccds(return_fname=True):
    img_raw, name_raw = next(ccds_obj_raw)
    img_trimmed, name_trimmed = next(ccds_obj_trimmed)
    img_unbiased, name_unbiased = next(ccds_obj_unbiased)
    assert name == name_raw == name_trimmed == name_unbiased
    
    show_images([img_raw, img_trimmed, img_unbiased, img], ["raw", "trimmed", "unbiased", "reduced"])
    plt.tight_layout()
    plt.gcf().suptitle(img.header["filter2"] + " " + name)
    savefig("reduction_frames." + name)
    plt.show()

In [ ]:
ccds_obj_raw = get_images(imgs_std, ccd_kwargs={"unit": "adu"}, return_fname=True)
ccds_obj_trimmed = get_images(allimgs_trimmed[-2], return_fname=True)
ccds_obj_unbiased = get_images(allimgs_trimmed[-2], return_fname=True)

for img, name in imgs_std_reduced.ccds(return_fname=True):
    img_raw, name_raw = next(ccds_obj_raw)
    img_trimmed, name_trimmed = next(ccds_obj_trimmed)
    img_unbiased, name_unbiased = next(ccds_obj_unbiased)
    assert name == name_raw == name_trimmed == name_unbiased
    
    show_images([img_raw, img_trimmed, img_unbiased, img], ["raw", "trimmed", "unbiased", "reduced"])
    plt.tight_layout()
    plt.gcf().suptitle(img.header["filter2"] + " " + name)
    savefig("reduction_frames.std." + name)
    plt.show()